# Risk Models: Volatility Regime Classification

This notebook demonstrates a practical regime framework for crypto risk monitoring.

## What this notebook teaches
- Standardized FinUties API setup for community notebooks
- Rolling volatility estimation from price returns
- Regime classification using data-driven thresholds

In [ ]:
from pathlib import Path
import os

import matplotlib.pyplot as plt
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv(Path.cwd().parent / ".env")
API_ORIGIN = os.getenv("FINUTIES_API_ORIGIN", "https://data.finuties.com").rstrip("/")
API_KEY = os.getenv("FINUTIES_API_KEY", "").strip()
if not API_KEY:
    raise ValueError("Missing FINUTIES_API_KEY in notebooks/.env")
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

In [ ]:
response = requests.get(
    f"{API_ORIGIN}/api/v1/data/crypto/prices",
    headers=HEADERS,
    params={"limit": 400},
    timeout=30,
)
response.raise_for_status()
payload = response.json()
rows = payload if isinstance(payload, list) else payload.get("items") or payload.get("data") or []

df = pd.DataFrame(rows)
if not {"timestamp", "price_usd"}.issubset(df.columns):
    raise ValueError(f"Unexpected risk-model schema: {list(df.columns)}")

In [ ]:
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df["price_usd"] = pd.to_numeric(df["price_usd"], errors="coerce")
df = df.dropna(subset=["timestamp", "price_usd"]).sort_values("timestamp").reset_index(drop=True)

df["ret"] = df["price_usd"].pct_change().fillna(0.0)
df["vol_30"] = df["ret"].rolling(30, min_periods=10).std().fillna(0.0)
threshold = df["vol_30"].quantile(0.70)
df["regime"] = df["vol_30"].gt(threshold).map({True: "high_vol", False: "low_vol"})

df[["timestamp", "price_usd", "vol_30", "regime"]].tail(12)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].plot(df["timestamp"], df["price_usd"], color="#1f77b4", linewidth=1.7)
axes[0].set_title("Price Path")
axes[0].set_xlabel("Timestamp")
axes[0].set_ylabel("Price (USD)")

axes[1].plot(df["timestamp"], df["vol_30"] * 100, color="#d62728", linewidth=1.7, label="Rolling vol (30)")
axes[1].axhline(threshold * 100, color="#444444", linestyle="--", linewidth=1, label="70th percentile")
axes[1].set_title("Volatility Regime Boundary")
axes[1].set_xlabel("Timestamp")
axes[1].set_ylabel("Volatility (%)")
axes[1].legend()

plt.tight_layout()
plt.show()

## Interpretation and caveats

- Regimes are relative to the selected sample, not absolute risk labels.
- Threshold choices should be backtested for your use case.

## Community extension ideas

- Add three regimes (low/medium/high) with tercile thresholds.
- Compare realized and implied volatility when options data is available.